# Random Forest

In [111]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

from sklearn import datasets
from sklearn import linear_model
from sklearn import naive_bayes
from sklearn import neighbors
from sklearn import linear_model
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor

In [112]:
# WARNINGS 02/06/2018:
# I updated the preprocessing notebook to keep the date as a feature. Run preprocessing again. 
# (Non provate a fare join qui col train.csv perché incasina tutti gli indici)
data = pd.read_csv('preprocessed_train.csv')
data.head()

# random row
data.loc[4342]

AssortmentType                With Non-Food Department
CloudCover                                           5
Date                                        2017-06-17
Events                                            Rain
HasPromotions                                        0
IsHoliday                                            0
Max_Dew_PointC                                      13
Max_Gust_SpeedKm_h                                  26
Max_Humidity                                        94
Max_Sea_Level_PressurehPa                         1021
Max_TemperatureC                                    20
Max_VisibilityKm                                    31
Max_Wind_SpeedKm_h                                  26
Mean_Dew_PointC                                      9
Mean_Humidity                                       64
Mean_Sea_Level_PressurehPa                        1018
Mean_TemperatureC                                   16
Mean_VisibilityKm                                   13
Mean_Wind_

### Some preprocessing again...

In [113]:
data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')

# Get the index for which the rolling fails (region 2, from 03/01/2018 to 04/02/2018)
start = pd.to_datetime('3/01/2018', format='%d/%m/%Y')
stop = pd.to_datetime('4/02/2018', format='%d/%m/%Y')

to_eliminate = data.loc[data.Region == 2]
to_eliminate = to_eliminate[to_eliminate.Date > start]
to_eliminate = to_eliminate[to_eliminate.Date < stop]

# I will remove these rows later...

In [114]:
to_eliminate

,AssortmentType,CloudCover,Date,Events,HasPromotions,IsHoliday,Max_Dew_PointC,Max_Gust_SpeedKm_h,Max_Humidity,Max_Sea_Level_PressurehPa,...,Month,Week,Quarter,IsOpen_yesterday,IsOpen_tomorrow,IsHoliday_yesterday,IsHoliday_tomorrow,NumberOfSales_yesterday,NumberOfSales_lastweek,NumberOfSales_lastmonth
2695,General,7.0,2018-01-05,Fog-Rain,0,0,2,35.0,100,1034,...,1,1,1,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2696,General,6.0,2018-01-06,Rain-Snow,0,0,2,42.0,100,1034,...,1,1,1,1.0,0.0,0.0,0.0,3597.0,3597.0,3597.0
2697,General,6.0,2018-01-08,Snow,1,0,-1,24.0,93,1034,...,1,2,1,0.0,0.0,0.0,1.0,0.0,7250.0,7250.0
2698,General,7.0,2018-01-10,Rain-Snow,1,0,1,14.0,100,1031,...,1,2,1,0.0,1.0,1.0,0.0,0.0,13528.0,13528.0
2699,General,7.0,2018-01-11,Rain,1,0,4,76.0,100,1030,...,1,2,1,1.0,1.0,0.0,0.0,5318.0,18846.0,18846.0
2700,General,6.0,2018-01-12,Rain,1,0,7,87.0,93,1032,...,1,2,1,1.0,1.0,0.0,0.0,4861.0,23707.0,23707.0
2701,General,5.0,2018-01-13,Rain,0,0,9,89.0,93,1018,...,1,2,1,1.0,0.0,0.0,0.0,4368.0,24478.0,28075.0
2702,General,6.0,2018-01-15,Rain,1,0,-1,71.0,81,1026,...,1,3,1,0.0,1.0,0.0,0.0,0.0,24939.0,32189.0
2703,General,0.0,2018-01-16,NaN,1,0,-3,23.0,93,1022,...,1,3,1,1.0,1.0,0.0,0.0,5410.0,24071.0,37599.0
2704,General,5.0,2018-01-17,Rain-Thunderstorm,1,0,3,80.0,93,1021,...,1,3,1,1.0,1.0,0.0,0.0,4392.0,28463.0,41991.0


In [115]:
data = pd.get_dummies(data, columns=['StoreType'], prefix='StoreType')
data = pd.get_dummies(data, columns=['AssortmentType'], prefix='AssortmentType')


In [116]:
# Keep most important features for our target (see FeatureSelection notebook for details)
# I kept also 'Date' because I need it, I will remove it then

selected_features=[
    'Date',
    'NumberOfSales',
    'NumberOfSales_lastmonth', 
    'HasPromotions', 
    'NumberOfSales_yesterday', 
    'NumberOfSales_lastweek',
    'IsOpen_yesterday',
    'DayOfWeek',
    'NearestCompetitor',
    'Week',
    'StoreID',
    'IsHoliday_tomorrow',
    'StoreType_Hyper Market',
    'Region_PopulationK',
    'Month',
    'IsOpen_tomorrow',
    'Max_TemperatureC',
    'Min_Humidity',
    'Min_Sea_Level_PressurehPa',
    'Mean_Wind_SpeedKm_h',
    'Mean_Humidity']
df = data[selected_features]
df.head()

,Date,NumberOfSales,NumberOfSales_lastmonth,HasPromotions,NumberOfSales_yesterday,NumberOfSales_lastweek,IsOpen_yesterday,DayOfWeek,NearestCompetitor,Week,...,IsHoliday_tomorrow,StoreType_Hyper Market,Region_PopulationK,Month,IsOpen_tomorrow,Max_TemperatureC,Min_Humidity,Min_Sea_Level_PressurehPa,Mean_Wind_SpeedKm_h,Mean_Humidity
0,2016-04-02,11129.0,186345.0,0,0.0,44106.0,0.0,5,326,13,...,0.0,1,2770,4,0.0,6,34,1005,13,64
1,2016-04-05,6981.0,182020.0,0,0.0,40680.0,0.0,1,326,14,...,0.0,1,2770,4,1.0,8,22,1009,18,49
2,2016-04-06,5222.0,189001.0,0,6981.0,39072.0,1.0,2,326,14,...,0.0,1,2770,4,1.0,7,29,1013,19,53
3,2016-04-07,6509.0,184113.0,0,5222.0,35239.0,1.0,3,326,14,...,0.0,1,2770,4,1.0,10,28,1008,16,52
4,2016-04-08,6270.0,181603.0,0,6509.0,29841.0,1.0,4,326,14,...,0.0,1,2770,4,1.0,4,50,1007,16,63


### Recover missing values

In [117]:
# Maybe we could improve the performance handling these missing values in better ways

null_cols = []
print('Features with NaN:')
for col in df.columns:
    if df[col].isnull().values.any():
        print(col)
        null_cols.append(col)
        
# getting indices (rows) of all NaN values
inds = pd.isnull(df).any(1).nonzero()[0]

# drop all the rows with NaN values
df = df.drop(df.index[inds])

# checking shapes
print('df: ' + str(df.shape))

Features with NaN:
NumberOfSales_lastmonth
NumberOfSales_yesterday
NumberOfSales_lastweek
IsOpen_yesterday
IsHoliday_tomorrow
IsOpen_tomorrow
df: (423510, 21)


### Split dataset in train and validation (che io chiamo test)

In [118]:
def split(df, train_fraction):
    mindate = df.Date.min()
    maxdate = df.Date.max()
    splitdate = mindate + (maxdate - mindate) * train_fraction
    train = df[df.Date < splitdate]
    test = df[df.Date >= splitdate]
    return train, test


In [119]:
# Save dataset with 'Date' feature and all the rows for the rolling 
df_dates = df

# Remove from the dataset the rows from which starting a rolling generates NaN values
df = df.drop(to_eliminate.index)

train_fraction = 21/24

df_train, df_test = split(df, train_fraction)

In [120]:
# Put rows back to train set 
df_train = pd.concat([df_dates.loc[to_eliminate.index], df_train])

In [121]:
# remove Date
df_train = df_train.drop('Date', axis=1)
df_test = df_test.drop('Date', axis=1)

In [154]:
# create dataset and target
ytrain = df_train['NumberOfSales']
Xtrain = df_train.drop('NumberOfSales',axis=1)

ytest = df_test['NumberOfSales']
Xtest = df_test.drop('NumberOfSales',axis=1)


## Compute the Random Forest

In [61]:
forest = RandomForestRegressor(n_estimators=250, random_state=0, n_jobs=-1)
forest.fit(Xtrain, ytrain)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=250, n_jobs=-1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [62]:
# How much good is the model?
forest.score(Xtest,ytest)

0.8288397723048758

## Run the prediction without knowing NumberOfSales_[lastWeek, lastmonth, yesterday]

In [155]:
# FOR DEBUGGING PORPOSE keep only one region in the test set
Xtest = Xtest[Xtest.Region_PopulationK == 2770]

In [156]:
# Forget these NumberOfSales values :(

Xtest['NumberOfSales_yesterday'] = np.NaN
Xtest['NumberOfSales_lastweek'] = np.NaN
Xtest['NumberOfSales_lastmonth'] = np.NaN

In [157]:
# remove all the values about NumberOfSales of the test set
df_dates.loc[Xtest.index,'NumberOfSales_yesterday'] = np.NaN
df_dates.loc[Xtest.index,'NumberOfSales_lastweek'] = np.NaN
df_dates.loc[Xtest.index,'NumberOfSales_lastmonth'] = np.NaN
df_dates.loc[Xtest.index,'NumberOfSales'] = np.NaN

In [159]:
import datetime 
import dateutil.relativedelta

# prepare the dataframe where store the prediction
y_predicted = pd.DataFrame(np.nan, index=Xtest.index, columns=['NumberOfSales_predicted'])
    
for index, row in Xtest.iterrows():
    
    print('working on ' , index)
    
    # get from train set all the rows about this store
    temp = df_dates.loc[df_dates.StoreID == row['StoreID']]
    
    # get the date of the 'row' 
    date = temp.loc[index]
    date = date['Date']
    
    # get all the data from the last month
    splitdate = date + dateutil.relativedelta.relativedelta(days=-31)
    temp = temp[temp.Date >= splitdate]
    temp = temp[temp.Date <= date]
     
    
    # switch index to timestamps to make this easier
    oldindex = temp.index
    temp.index = temp['Date']
    
    temp=temp.sort_index()
    
    temp['NumberOfSales_yesterday'] = temp.NumberOfSales.rolling(window='1d',closed='left', min_periods=1).sum()
    temp['NumberOfSales_lastweek'] = temp.NumberOfSales.rolling(window='7d',closed='left', min_periods=1).sum()
    temp['NumberOfSales_lastmonth'] = temp.NumberOfSales.rolling(window='30d',closed='left', min_periods=1).sum()

    temp.NumberOfSales_yesterday = temp.NumberOfSales_yesterday.fillna(0)

    # put it back in the dataframe
    temp.index = oldindex
    
    # put the result in 'row'
    row['NumberOfSales_yesterday'] = temp.loc[index,'NumberOfSales_yesterday']
    row['NumberOfSales_lastweek'] = temp.loc[index,'NumberOfSales_lastweek']
    row['NumberOfSales_lastmonth'] = temp.loc[index,'NumberOfSales_lastmonth']

    # predict NumberOfSales of today
    temp.loc[index,'NumberOfSales'] = forest.predict(row.values.reshape(1, -1))
    
    # at the next iter my I can use this prediction for a new rolling
    df_dates.loc[index] = temp.loc[index]

    y_predicted.loc[index] = temp.loc[index,'NumberOfSales'] 



working on  497
working on  498
working on  499
working on  500
working on  501
working on  502
working on  503
working on  504
working on  505
working on  506
working on  507
working on  508
working on  509
working on  510
working on  511
working on  512
working on  513
working on  514
working on  515
working on  516
working on  517
working on  518
working on  519
working on  520
working on  521
working on  522
working on  523
working on  524
working on  525
working on  526
working on  527
working on  528
working on  529
working on  530
working on  531
working on  532
working on  533
working on  534
working on  535
working on  536
working on  537
working on  538
working on  539
working on  540
working on  541
working on  542
working on  543
working on  544
working on  545
working on  546
working on  547
working on  548
working on  549
working on  550
working on  551
working on  552
working on  553
working on  554
working on  555
working on  556
working on  557
working on  558
working 

working on  77445
working on  77446
working on  77447
working on  77448
working on  77449
working on  77450
working on  77451
working on  77452
working on  77453
working on  77454
working on  77455
working on  77456
working on  77457
working on  77458
working on  77459
working on  77460
working on  77461
working on  77462
working on  77463
working on  77464
working on  77465
working on  77466
working on  77467
working on  77468
working on  77469
working on  77470
working on  77471
working on  77472
working on  77473
working on  77474
working on  77475
working on  77476
working on  77477
working on  77478
working on  77479
working on  77480
working on  77481
working on  77482
working on  77483
working on  77484
working on  77485
working on  77486
working on  77487
working on  77488
working on  77489
working on  77490
working on  77491
working on  77492
working on  77493
working on  77494
working on  77495
working on  77496
working on  81496
working on  81497
working on  81498
working on

working on  117643
working on  117644
working on  117645
working on  117646
working on  117647
working on  117648
working on  117649
working on  117650
working on  117651
working on  117652
working on  117653
working on  117654
working on  117655
working on  117656
working on  117657
working on  117658
working on  117659
working on  117660
working on  117661
working on  117662
working on  117663
working on  117664
working on  117665
working on  117666
working on  117667
working on  117668
working on  117669
working on  117670
working on  117671
working on  117672
working on  117673
working on  117674
working on  117675
working on  117676
working on  117677
working on  117678
working on  117679
working on  117680
working on  117681
working on  117682
working on  117683
working on  117684
working on  117685
working on  117686
working on  117687
working on  117688
working on  117689
working on  117690
working on  117691
working on  117692
working on  117693
working on  117694
working on  

working on  139120
working on  139121
working on  139122
working on  139123
working on  139124
working on  139125
working on  139126
working on  139127
working on  139128
working on  139129
working on  139130
working on  139131
working on  139132
working on  139133
working on  139134
working on  139135
working on  139136
working on  139137
working on  139138
working on  139139
working on  139140
working on  139141
working on  139142
working on  139143
working on  139144
working on  139145
working on  139146
working on  139147
working on  139148
working on  139149
working on  139150
working on  139151
working on  139152
working on  139153
working on  139154
working on  139155
working on  139156
working on  139157
working on  139158
working on  139159
working on  139160
working on  139161
working on  139162
working on  139163
working on  139164
working on  139165
working on  139166
working on  139167
working on  139168
working on  139169
working on  139170
working on  139171
working on  

working on  157877
working on  157878
working on  157879
working on  157880
working on  157881
working on  157882
working on  157883
working on  157884
working on  157885
working on  157886
working on  157887
working on  157888
working on  157889
working on  162050
working on  162051
working on  162052
working on  162053
working on  162054
working on  162055
working on  162056
working on  162057
working on  162058
working on  162059
working on  162060
working on  162061
working on  162062
working on  162063
working on  162064
working on  162065
working on  162066
working on  162067
working on  162068
working on  162069
working on  162070
working on  162071
working on  162072
working on  162073
working on  162074
working on  162075
working on  162076
working on  162077
working on  162078
working on  162079
working on  162080
working on  162081
working on  162082
working on  162083
working on  162084
working on  162085
working on  162086
working on  162087
working on  162088
working on  

...

working on  214942
working on  214943
working on  214944
working on  214945
working on  214946
working on  214947
working on  214948
working on  214949
working on  214950
working on  214951
working on  214952
working on  214953
working on  214954
working on  214955
working on  214956
working on  214957
working on  214958
working on  214959
working on  214960
working on  214961
working on  214962
working on  214963
working on  214964
working on  214965
working on  214966
working on  214967
working on  214968
working on  214969
working on  214970
working on  214971
working on  214972
working on  214973
working on  214974
working on  214975
working on  214976
working on  214977
working on  214978
working on  214979
working on  214980
working on  214981
working on  214982
working on  214983
working on  214984
working on  214985
working on  214986
working on  214987
working on  214988
working on  214989
working on  214990
working on  223648
working on  223649
working on  223650
working on  

working on  265666
working on  265667
working on  265668
working on  265669
working on  265670
working on  265671
working on  265672
working on  269835
working on  269836
working on  269837
working on  269838
working on  269839
working on  269840
working on  269841
working on  269842
working on  269843
working on  269844
working on  269845
working on  269846
working on  269847
working on  269848
working on  269849
working on  269850
working on  269851
working on  269852
working on  269853
working on  269854
working on  269855
working on  269856
working on  269857
working on  269858
working on  269859
working on  269860
working on  269861
working on  269862
working on  269863
working on  269864
working on  269865
working on  269866
working on  269867
working on  269868
working on  269869
working on  269870
working on  269871
working on  269872
working on  269873
working on  269874
working on  269875
working on  269876
working on  269877
working on  269878
working on  269879
working on  

working on  309990
working on  309991
working on  309992
working on  309993
working on  309994
working on  309995
working on  309996
working on  309997
working on  309998
working on  309999
working on  310000
working on  310001
working on  310002
working on  310003
working on  310004
working on  310005
working on  310006
working on  310007
working on  310008
working on  310009
working on  310010
working on  310011
working on  310012
working on  310013
working on  310014
working on  318690
working on  318691
working on  318692
working on  318693
working on  318694
working on  318695
working on  318696
working on  318697
working on  318698
working on  318699
working on  318700
working on  318701
working on  318702
working on  318703
working on  318704
working on  318705
working on  318706
working on  318707
working on  318708
working on  318709
working on  318710
working on  318711
working on  318712
working on  318713
working on  318714
working on  318715
working on  318716
working on  

working on  365741
working on  365742
working on  365743
working on  365744
working on  365745
working on  365746
working on  365747
working on  365748
working on  365749
working on  365750
working on  365751
working on  365752
working on  365753
working on  365754
working on  365755
working on  365756
working on  365757
working on  365758
working on  365759
working on  365760
working on  365761
working on  365762
working on  365763
working on  365764
working on  365765
working on  365766
working on  365767
working on  365768
working on  365769
working on  365770
working on  365771
working on  365772
working on  365773
working on  365774
working on  365775
working on  365776
working on  365777
working on  365778
working on  365779
working on  365780
working on  365781
working on  365782
working on  365783
working on  370991
working on  370992
working on  370993
working on  370994
working on  370995
working on  370996
working on  370997
working on  370998
working on  370999
working on  

working on  411047
working on  411048
working on  411049
working on  411050
working on  411051
working on  411052
working on  411053
working on  411054
working on  411055
working on  411056
working on  411057
working on  411058
working on  411059
working on  411060
working on  411061
working on  411062
working on  411063
working on  411064
working on  411065
working on  411066
working on  411067
working on  411068
working on  411069
working on  411070
working on  411071
working on  411072
working on  411073
working on  411074
working on  411075
working on  411076
working on  411077
working on  411078
working on  411079
working on  411080
working on  411081
working on  411082
working on  411083
working on  411084
working on  411085
working on  411086
working on  411087
working on  411088
working on  411089
working on  411090
working on  411091
working on  411092
working on  411093
working on  411094
working on  411095
working on  411096
working on  411097
working on  411098
working on  

## Score  ( deprecated ) - use bip ufficial method

In [166]:
df1 = df_test.join(y_predicted)

In [167]:
# FOR DEBUGGING PORPOSE keep only one region in the test set
df1 = df1[df1.Region_PopulationK == 2770]

In [168]:
# group by month & region 
result=df1.groupby(['Month','Region_PopulationK'], as_index=False).agg({"NumberOfSales": "sum","NumberOfSales_predicted":"sum"})

# % error
result['% Error']=abs(result['NumberOfSales']-result['NumberOfSales_predicted'])/result['NumberOfSales']*100

In [169]:
result

,Month,Region_PopulationK,NumberOfSales,NumberOfSales_predicted,% Error
0,1,2770,7285029.0,9559860.240,31.226111
1,2,2770,6747479.0,9735660.488,44.285895
2,11,2770,344595.0,323940.384,5.993882
3,12,2770,8750110.0,9213796.548,5.299208


In [171]:
result['Error'].sum()/result.count().max()

KeyError: 'Error'